In [1]:
import sys
sys.path.append("../..")

import time
from time import sleep
from datetime import datetime, timedelta
import threading

import MetaTrader5 as mt5
import pandas as pd
from IPython.display import display
from lightweight_charts import JupyterChart
from IPython.display import clear_output, display

from sesto.fractal import high_low_finder
from sesto.utils import get_price_at_pnl, calculate_commission, calculate_position_size, get_pnl_at_price, convert_lots_to_usd, calculate_trade_volume, convert_usd_to_lots
from sesto.metatrader.business import get_positions, send_market_order, modify_sl_tp, get_order_from_ticket, get_deal_from_ticket
from sesto.metatrader.data import fetch_data_pos
from sesto.constants import CRYPTOCURRENCIES, CURRENCY_PAIRS, TIMEZONE, MT5Timeframe
from sesto.telegram import TelegramSender

MetaTrader5 initialized successfully
MetaTrader 5 initialized successfully.


In [2]:
Telegram = TelegramSender()

In [3]:
PAIRS = CRYPTOCURRENCIES + CURRENCY_PAIRS
MAIN_TIMEFRAME = MT5Timeframe.M15

TP_PNL_MULTIPLIER = 0.5
SL_PNL_MULTIPLIER = -0.25
LEVERAGE = 500
DEVIATION = 20
VOLUME = 0.2

TRAILING_STOP_STEPS = [
    {'trigger_pnl_multiplier': 4.00, 'new_sl_pnl_multiplier': 3.50},
    {'trigger_pnl_multiplier': 3.50, 'new_sl_pnl_multiplier': 3.00},
    {'trigger_pnl_multiplier': 3.00, 'new_sl_pnl_multiplier': 2.75},
    {'trigger_pnl_multiplier': 2.75, 'new_sl_pnl_multiplier': 2.50},
    {'trigger_pnl_multiplier': 2.50, 'new_sl_pnl_multiplier': 2.25},
    {'trigger_pnl_multiplier': 2.25, 'new_sl_pnl_multiplier': 2.00},
    {'trigger_pnl_multiplier': 2.00, 'new_sl_pnl_multiplier': 1.75},
    {'trigger_pnl_multiplier': 1.75, 'new_sl_pnl_multiplier': 1.50},
    {'trigger_pnl_multiplier': 1.50, 'new_sl_pnl_multiplier': 1.25},
    {'trigger_pnl_multiplier': 1.25, 'new_sl_pnl_multiplier': 1.00},
    {'trigger_pnl_multiplier': 1.00, 'new_sl_pnl_multiplier': 0.75},
    {'trigger_pnl_multiplier': 0.75, 'new_sl_pnl_multiplier': 0.45},
    {'trigger_pnl_multiplier': 0.50, 'new_sl_pnl_multiplier': 0.22},
    {'trigger_pnl_multiplier': 0.25, 'new_sl_pnl_multiplier': 0.12},
    {'trigger_pnl_multiplier': 0.12, 'new_sl_pnl_multiplier': 0.05},
    {'trigger_pnl_multiplier': 0.06, 'new_sl_pnl_multiplier': 0.025},
]

In [4]:
def calculate_trade_capital(symbol, volume_lots, leverage, price_open):
    position_size_usd = convert_lots_to_usd(symbol, volume_lots, price_open)
    capital_used = position_size_usd / leverage
    return capital_used

In [5]:
def have_open_positions_in_symbol(symbol):
    positions = get_positions()
    return symbol in positions['symbol'].values

In [6]:
# dict to store trades, keys are position tickets, values are the entire position object
trades = {}

In [7]:
def check_fractal_signals():
    """
    Continuously monitors for fractal signals and places market orders accordingly.
    """
    while True:
        try:
            for pair in PAIRS:
                if have_open_positions_in_symbol(pair):
                    continue

                if pair in CURRENCY_PAIRS:
                    # Check whether the market is open
                    tick = mt5.symbol_info_tick(pair)
                    if tick is not None:
                        tick_time = datetime.fromtimestamp(tick.time, tz=TIMEZONE)
                        current_time = datetime.now(TIMEZONE)
                        time_difference = current_time - tick_time

                        if time_difference > timedelta(minutes=5):
                            continue
                    else:
                        continue

                df = fetch_data_pos(pair, MAIN_TIMEFRAME, 50)
                if df is None or df.empty:
                    print(f"No data fetched for {pair}. Skipping...")
                    continue

                df['fractal'] = high_low_finder(df)
                df['symbol'] = pair

                price = mt5.symbol_info_tick(pair).ask
                if price is None:
                    print(f"Failed to retrieve ask price for {pair}.")
                    continue

                new_trade_capital = calculate_trade_capital(pair, VOLUME, LEVERAGE, price)
                trade_size = calculate_position_size(new_trade_capital, LEVERAGE)
                commission = calculate_commission(position_size_usd=trade_size, pair=pair)

                last_row = df.iloc[-2]
                current_time = datetime.now(TIMEZONE).replace(microsecond=0)

                if last_row['fractal'] in ['top', 'bottom']:
                    sl = get_price_at_pnl(
                        pnl_multiplier=SL_PNL_MULTIPLIER,
                        order_commission=commission,
                        position_size_usd=trade_size,
                        leverage=LEVERAGE,
                        entry_price=price,
                        type='long' if last_row['fractal'] == 'bottom' else 'short'
                    )
                    order = send_market_order(
                        symbol=pair,
                        volume=VOLUME,
                        order_type='sell' if last_row['fractal'] == 'top' else 'buy',
                        sl=sl,
                        deviation=DEVIATION,
                        type_filling=mt5.ORDER_FILLING_FOK
                    )
                    if order is not None:
                        trade_info = {
                            'event': 'trade_opened',
                            'symbol': pair,
                            'entry_condition': f"{last_row['fractal'].upper()} FRACTAL DETECTED",
                            'type': 'long' if last_row['fractal'] == 'bottom' else 'short',
                            'row_close': f"${last_row['close']:.5f}",
                            'last_tick_price': f"${price:.5f}",
                            'entry_price': f"${order.price:.5f}",
                            'sl': f"${sl:.5f}",
                            'pnl_at_sl': f"${get_pnl_at_price(sl, price, trade_size, LEVERAGE, 'long' if last_row['fractal'] == 'bottom' else 'short'):.5f}",
                            'volume': VOLUME,
                            'trade_capital': f"${new_trade_capital:.5f}",
                            'commission': f"${commission:.5f}",
                            'trade_size': f"${trade_size:.5f}",
                            'time': current_time.isoformat()
                        }
                        Telegram.send_json_message(trade_info)
                        print(f"Order placed successfully for {pair}: {trade_info}")
                    else:
                        trade_info = {
                            'event': 'trade_failed_to_open',
                            'symbol': pair,
                            'entry_condition': f"{last_row['fractal'].upper()} FRACTAL DETECTED",
                            'type': 'long' if last_row['fractal'] == 'bottom' else 'short',
                            'row_close': f"${last_row['close']:.5f}",
                            'last_tick_price': f"${price:.5f}",
                            'sl': f"${sl:.5f}",
                            'pnl_at_sl': f"${get_pnl_at_price(sl, price, trade_size, LEVERAGE, 'long' if last_row['fractal'] == 'bottom' else 'short'):.5f}",
                            'volume': VOLUME,
                            'trade_capital': f"${new_trade_capital:.5f}",
                            'commission': f"${commission:.5f}",
                            'trade_size': f"${trade_size:.5f}",
                            'time': current_time.isoformat()
                        }
                        Telegram.send_json_message(trade_info)
            # Calculate next run time aligned to the next 15-minute interval
            now = datetime.now(TIMEZONE)
            next_run = (now + timedelta(minutes=15 - now.minute % 15)).replace(second=0, microsecond=0)
            sleep_time = (next_run - now).total_seconds() + 2  # Add 2 seconds buffer

            print(f"{datetime.now(tz=TIMEZONE)} - Waiting for {sleep_time} seconds until next 15-minute check.")
            sleep(sleep_time)
        
        except Exception as e:
            print(f"Exception in check_fractal_signals: {e}")
            Telegram.send_json_message(e)
            sleep(10)  # Wait before retrying in case of error

def monitor_open_trades():
    """
    Continuously monitors open trades, detects closed trades, and sends notifications.
    Utilizes a cached state to detect changes in open positions.
    """
    cached_trades = {}

    while True:
        try:
            current_time = datetime.now(TIMEZONE).replace(microsecond=0)

            positions = get_positions()
            if positions.empty:
                positions = pd.DataFrame(columns=[
                    'ticket', 'time', 'time_msc', 'time_update', 'time_update_msc', 'type',
                    'magic', 'identifier', 'reason', 'volume', 'price_open', 'sl', 'tp',
                    'price_current', 'swap', 'profit', 'symbol', 'comment', 'external_id'
                ])

            positions['time'] = pd.to_datetime(positions['time'], unit='s', utc=True)
            positions['time_update'] = pd.to_datetime(positions['time_update'], unit='s', utc=True)

            # Detect closed trades by comparing cached_trades with current positions
            current_tickets = set(positions['ticket'].values)
            cached_tickets = set(cached_trades.keys())

            # Identify closed tickets
            closed_tickets = cached_tickets - current_tickets

            for ticket in closed_tickets:
                trade = cached_trades.pop(ticket)
                # Fetch and send closed trade details
                sleep(2)
                closed_order = get_order_from_ticket(ticket)
                if closed_order:
                    closed_order['event'] = 'trade_closed_order'
                    # Telegram.send_json_message(closed_order)

                closed_deal = get_deal_from_ticket(ticket)
                if closed_deal:
                    closed_deal['event'] = 'trade_closed_deal'
                    Telegram.send_json_message(closed_deal)

            # Update cached_trades with current positions
            for index, position in positions.iterrows():
                cached_trades[position.ticket] = position

                # Check if the position ticket exists in trades dict
                if position.ticket not in trades:
                    trades[position.ticket] = position
                else:
                    # Check if position has changed
                    if not trades[position.ticket].equals(position):
                        trades[position.ticket] = position

                # Calculate the actual capital used for this trade
                trade_capital = calculate_trade_capital(
                    position.symbol, position.volume, LEVERAGE, position.price_open
                )
                trade_size = calculate_position_size(trade_capital, LEVERAGE)
                commission = calculate_commission(position_size_usd=trade_size, pair=position.symbol)
                current_pnl_percentage = (position.profit / trade_capital) * 100
                current_sl_pnl = get_pnl_at_price(
                    position.sl, position.price_open, trade_size, LEVERAGE,
                    'long' if position.type == 0 else 'short'
                )

                for trailing_step in TRAILING_STOP_STEPS:
                    trigger_pnl_multiplier = trailing_step['trigger_pnl_multiplier']
                    new_sl_pnl_multiplier = trailing_step['new_sl_pnl_multiplier']

                    trigger_price = get_price_at_pnl(
                        pnl_multiplier=trigger_pnl_multiplier,
                        order_commission=commission,
                        position_size_usd=trade_size,
                        leverage=LEVERAGE,
                        entry_price=position.price_open,
                        type='long' if position.type == 0 else 'short'
                    )
                    new_sl = get_price_at_pnl(
                        pnl_multiplier=new_sl_pnl_multiplier,
                        order_commission=commission,
                        position_size_usd=trade_size,
                        leverage=LEVERAGE,
                        entry_price=position.price_open,
                        type='long' if position.type == 0 else 'short'
                    )

                    trigger_pnl = get_pnl_at_price(
                        trigger_price,
                        position.price_open,
                        trade_size,
                        LEVERAGE,
                        'long' if position.type == 0 else 'short'
                    )

                    pnl_at_new_sl = get_pnl_at_price(
                        new_sl,
                        position.price_open,
                        trade_size,
                        LEVERAGE,
                        'long' if position.type == 0 else 'short'
                    )

                    if position.profit is not None and trigger_pnl is not None and position.profit >= trigger_pnl:
                        if position.sl is not None and new_sl is not None:
                            if (position.type == 0 and new_sl > position.sl) or (position.type == 1 and new_sl < position.sl):
                                sl_info = {
                                    'event': 'trailing_stop_triggered',
                                    'position_data': {
                                        'symbol': position.symbol,
                                        'trade_open_date': position.time.isoformat(),
                                        'type': 'long' if position.type == 0 else 'short',
                                        'entry_price': f"${position.price_open:.5f}",
                                        'current_price': f"${position.price_current:.5f}",
                                        'capital_used': f"${trade_capital:.5f}",
                                        'trade_size': f"${trade_size:.5f}",
                                        'deduced_volume': f"${calculate_trade_volume(position.price_open, position.price_current, position.profit, LEVERAGE):.5f}",
                                        'deduced_volume_lots': f"${convert_usd_to_lots(position.symbol, calculate_trade_volume(position.price_open, position.price_current, position.profit, LEVERAGE), position.price_current):.5f}",
                                        'commission': f"${commission:.5f}",
                                    },
                                    'trigger_data': {
                                        'trigger_price': f"${trigger_price:.5f}",
                                        'trigger_pnl': f"${trigger_pnl:.5f}",
                                        'trigger_pnl_percentage': f"{(trigger_pnl / trade_capital) * 100:.5f}%",
                                    },
                                    'current_pnl': {
                                        'current_pnl': f"${position.profit:.5f}",
                                        'current_pnl_percentage': f"{current_pnl_percentage:.5f}%",
                                    },
                                    'old_sl': {
                                        'old_sl': f"${position.sl:.5f}",
                                        'pnl_at_old_sl': f"${current_sl_pnl:.5f}",
                                        'old_sl_pnl_percentage': f"{(current_sl_pnl / trade_capital) * 100:.5f}%",
                                    },
                                    'new_sl': {
                                        'new_sl': f"${new_sl:.5f}",
                                        'pnl_at_new_sl': f"${pnl_at_new_sl:.5f}",
                                        'new_sl_pnl_percentage': f"{(pnl_at_new_sl / trade_capital) * 100:.5f}%",
                                    }
                                }

                                modify_request = modify_sl_tp(position.ticket, new_sl, position.tp)
                                if modify_request is not None:
                                    Telegram.send_json_message(sl_info)
                                break  # Exit the trailing steps loop after modification
                        # else:
                            # print(f"Warning: Stop Loss is None for position {position.ticket}")
                    # else:
                        # print(f"Warning: Profit or trigger PNL is None for position {position.ticket}")

            sleep(10)  # Polling interval for monitoring open trades

        except Exception as e:
            print(f"Exception in monitor_open_trades: {e}")
            Telegram.send_json_message(e)
            sleep(10)

In [8]:
if __name__ == "__main__":
    try:
        fractal_thread = threading.Thread(target=check_fractal_signals, daemon=True)
        monitor_thread = threading.Thread(target=monitor_open_trades, daemon=True)

        fractal_thread.start()
        monitor_thread.start()

        print(f"{datetime.now(tz=TIMEZONE)} - Threads started successfully.")

        while True:
            sleep(1)

    except KeyboardInterrupt:
        print(f"{datetime.now(tz=TIMEZONE)} - Program terminated by user.")
    except Exception as e:
        print(f"{datetime.now(tz=TIMEZONE)} - Unhandled exception: {e}")
        Telegram.send_json_message(e)

2024-10-05 19:56:11.499795+00:00 - Threads started successfully.Exception in monitor_open_trades: get_price_at_pnl() got an unexpected keyword argument 'order_commission'

Exception in check_fractal_signals: get_price_at_pnl() got an unexpected keyword argument 'order_commission'


Exception in check_fractal_signals: get_price_at_pnl() got an unexpected keyword argument 'order_commission'
Exception in monitor_open_trades: get_price_at_pnl() got an unexpected keyword argument 'order_commission'
Exception in check_fractal_signals: get_price_at_pnl() got an unexpected keyword argument 'order_commission'
Exception in monitor_open_trades: get_price_at_pnl() got an unexpected keyword argument 'order_commission'
Exception in check_fractal_signals: get_price_at_pnl() got an unexpected keyword argument 'order_commission'
Exception in monitor_open_trades: get_price_at_pnl() got an unexpected keyword argument 'order_commission'
2024-10-05 19:56:45.826063+00:00 - Program terminated by user.
